# Ensemble Learning: Bagging Tutorial

* ***We will use pima indian diabetes dataset to predict if a person has a diabetes or not based on certain features such as blood pressure, skin thickness, age etc. We will train a standalone model first and then use bagging ensemble technique to check how it can improve the performance of the model ***

In [12]:
import pandas as pd

df = pd.read_csv("E:\Jupyternotebook\ML practice\diabetes.csv")
df.head()

<>:3: SyntaxWarning: invalid escape sequence '\J'
<>:3: SyntaxWarning: invalid escape sequence '\J'
C:\Users\ASUS\AppData\Local\Temp\ipykernel_46640\2173122363.py:3: SyntaxWarning: invalid escape sequence '\J'
  df = pd.read_csv("E:\Jupyternotebook\ML practice\diabetes.csv")


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [13]:

df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [14]:

df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [15]:
##Checking imbalance in dataset (   .value_counts()  )
df.Outcome.value_counts()

Outcome
0    500
1    268
Name: count, dtype: int64

In [16]:
X = df.drop("Outcome",axis="columns")
y = df.Outcome

In [19]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled[:3]

array([[ 0.63994726,  0.84832379,  0.14964075,  0.90726993, -0.69289057,
         0.20401277,  0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575,  0.53090156, -0.69289057,
        -0.68442195, -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, -1.28821221, -0.69289057,
        -1.10325546,  0.60439732, -0.10558415]])

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify=y, random_state=10)

In [22]:
y_train.value_counts()

Outcome
0    375
1    201
Name: count, dtype: int64

# Train using stand alone model

In [23]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

scores = cross_val_score(DecisionTreeClassifier(), X, y, cv=5)
scores

array([0.66883117, 0.63636364, 0.68181818, 0.77777778, 0.7254902 ])

In [24]:
scores.mean()

0.6980561921738392

#  Train using Bagging

In [28]:
from sklearn.ensemble import BaggingClassifier

bag_model = BaggingClassifier(
    estimator=DecisionTreeClassifier(), 
    n_estimators=100, # How many sub grp of dataset
    max_samples=0.8, # use 80% of sample
    oob_score=True, # if any sample ismissed it willbe used in TEST
    random_state=0
)
bag_model.fit(X_train, y_train)
bag_model.oob_score_

0.7534722222222222

In [29]:
bag_model.score(X_test, y_test)

0.7760416666666666

In [30]:
# Using Cross Validation

bag_model = BaggingClassifier(
    estimator=DecisionTreeClassifier(), 
    n_estimators=100, 
    max_samples=0.8, 
    oob_score=True,
    random_state=0
)
scores = cross_val_score(bag_model, X, y, cv=5)
scores

array([0.75324675, 0.72727273, 0.74675325, 0.82352941, 0.73856209])

In [31]:

scores.mean()

0.7578728461081402

# Train using Random Forest

In [32]:
from sklearn.ensemble import RandomForestClassifier

scores = cross_val_score(RandomForestClassifier(n_estimators=50), X, y, cv=5)
scores.mean()

0.7682879212290976

# Exercise

*Download heart disease dataset heart.csv in Exercise folder and do following, (credits of dataset: https://www.kaggle.com/fedesoriano/heart-failure-prediction)

Load heart disease dataset in pandas dataframe
Remove outliers using Z score. Usual guideline is to remove anything that has Z score > 3 formula or Z score < -3
Convert text columns to numbers using label encoding and one hot encoding
Apply scaling
Build a classification model using support vector machine. Use standalone model as well as Bagging model and check if you see any difference in the performance.
Now use decision tree classifier. Use standalone model as well as Bagging and check if you notice any difference in performance
Comparing performance of svm and decision tree classifier figure out where it makes most sense to use bagging and why. Use internet to figure out in what conditions bagging works the best*

In [33]:
import pandas as  pd
df=pd.read_csv("E:\Jupyternotebook\ML practice\heart.csv")
df

<>:2: SyntaxWarning: invalid escape sequence '\J'
<>:2: SyntaxWarning: invalid escape sequence '\J'
C:\Users\ASUS\AppData\Local\Temp\ipykernel_46640\4186246176.py:2: SyntaxWarning: invalid escape sequence '\J'
  df=pd.read_csv("E:\Jupyternotebook\ML practice\heart.csv")


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [34]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


# Treat Outliers

In [35]:

# --------------------------- STEP 1: Clean Invalid Values ---------------------------- #

# Replace 0 in RestingBP with median (0 is not a valid blood pressure)
if (df['RestingBP']==0).any():
    restingbp_median=df[df["RestingBP"]>0]["RestingBP"].median()
    df['RestingBP']=df["RestingBP"].replace(0, restingbp_median)


# Replace 0 in Cholesterol with median (0 is not a valid cholesterol level)
if (df['Cholesterol'] == 0).any():
    cholesterol_median = df[df['Cholesterol'] > 0]['Cholesterol'].median()
    df['Cholesterol'] = df['Cholesterol'].replace(0, cholesterol_median)


# Replace negative Oldpeak values with 0 (ST depression cannot be negative)
df['Oldpeak']=df["Oldpeak"].apply(lambda x: max(x,0))

# --------------------------- STEP 2: Remove Outliers Using Z-Score Logic ---------------------------- #
def remove_high_outliers(df, column):
    """
    Removes rows where column value > mean + 3*std.
    Assumes only high outliers are problematic.
    """
    upper_limit = df[column].mean() + 3 * df[column].std()
    return df[df[column] <= upper_limit]


columns_to_clean = ['Cholesterol', 'Oldpeak', 'RestingBP']

for col in columns_to_clean:
    df = remove_high_outliers(df, col)


# ❌ MaxHR — Not necessary (high HR can be normal physiologically)
# ❌ FastingBS — Not necessary (binary 0/1, Z-score irrelevant)


# --------------------------- Final Cleaned Data ---------------------------- #
df_cleaned = df.copy()
print("Final cleaned shape:", df_cleaned.shape)




Final cleaned shape: (892, 12)


In [36]:
df.ChestPainType.unique()
df.RestingECG.unique()
df.ExerciseAngina.unique()
df.ST_Slope.unique()
df4 = df_cleaned.copy()
df4.ExerciseAngina.replace(
    {
        'N': 0,
        'Y': 1
    },
    inplace=True)

df4.ST_Slope.replace(
    {
        'Down': 1,
        'Flat': 2,
        'Up': 3
    },
    inplace=True
)

df4.RestingECG.replace(
    {
        'Normal': 1,
        'ST': 2,
        'LVH': 3
    },
    inplace=True)


df5 = pd.get_dummies(df4,drop_first=True)
df5.head()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_46640\4178187817.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df4.ExerciseAngina.replace(
C:\Users\ASUS\AppData\Local\Temp\ipykernel_46640\4178187817.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df4.ExerciseAngina.replace(
C:\Users\ASUS\AppDat

,Age,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA
0,40,140,289,0,1,172,0,0.0,3,0,True,True,False,False
1,49,160,180,0,1,156,0,1.0,2,1,False,False,True,False
2,37,130,283,0,2,98,0,0.0,3,0,True,True,False,False
3,48,138,214,0,1,108,1,1.5,2,1,False,False,False,False
4,54,150,195,0,1,122,0,0.0,3,0,True,False,True,False


In [37]:
X = df5.drop("HeartDisease",axis='columns')
y = df5.HeartDisease

X.head()

,Age,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA
0,40,140,289,0,1,172,0,0.0,3,True,True,False,False
1,49,160,180,0,1,156,0,1.0,2,False,False,True,False
2,37,130,283,0,2,98,0,0.0,3,True,True,False,False
3,48,138,214,0,1,108,1,1.5,2,False,False,False,False
4,54,150,195,0,1,122,0,0.0,3,True,False,True,False


In [38]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

array([[-1.42729841,  0.46864343,  1.08341545, ...,  2.06083935,
        -0.53587059, -0.23049661],
       [-0.47544996,  1.63372132, -1.31385173, ..., -0.48523918,
         1.8661222 , -0.23049661],
       [-1.74458123, -0.11389551,  0.95145579, ...,  2.06083935,
        -0.53587059, -0.23049661],
       ...,
       [ 0.37063755, -0.11389551, -2.3915223 , ..., -0.48523918,
        -0.53587059, -0.23049661],
       [ 0.37063755, -0.11389551, -0.08222823, ...,  2.06083935,
        -0.53587059, -0.23049661],
       [-1.63882029,  0.35213565, -1.42381812, ..., -0.48523918,
         1.8661222 , -0.23049661]])

In [39]:
from sklearn.model_selection import  train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_scaled,y,test_size=0.2,random_state=30)

#  Model

In [41]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

scores = cross_val_score(SVC(), X, y, cv=5)
scores.mean()

0.68723871696692

In [43]:
from sklearn.ensemble import BaggingClassifier

bag_model = BaggingClassifier(estimator=SVC(), n_estimators=100, max_samples=0.8, random_state=0)
scores = cross_val_score(bag_model, X, y, cv=5)
scores.mean()

0.6861276756010295

****
As you can see above, using bagging in case of SVM doesn't make much difference in terms of model accuracy. Bagging is effective when we have high variance and instable model such as decision tree. Let's explore how bagging changes the performance for a decision tree classifier. ***

# Train a model using decision tree and then using bagging

In [44]:
from sklearn.tree import DecisionTreeClassifier

scores = cross_val_score(DecisionTreeClassifier(random_state=0), X, y, cv=5)
scores.mean()

0.7465381959701212

# Use bagging now with decision tree

In [46]:
bag_model = BaggingClassifier(
    estimator=DecisionTreeClassifier(random_state=0), 
    n_estimators=100, 
    max_samples=0.9, 
    oob_score=True,
    random_state=0
)

scores = cross_val_score(bag_model, X, y, cv=5)
scores.mean()

0.8216809993095223

# Train a model using Random Forest which itself uses bagging underneath

In [47]:
from sklearn.ensemble import RandomForestClassifier

scores = cross_val_score(RandomForestClassifier(), X, y, cv=5)
scores.mean()

0.8284162952733665


Random forest gave even a better performance with 81.7% as score. Underneath it used bagging where it sampled not only data rows but also the columns (or features)